In [17]:
import numpy as np
import os
from tqdm.auto import tqdm, trange

In [2]:
def get_degrees(edges,node_count):
    edge_array = np.array(edges)
    degrees = [np.sum(edge_array==i) for i in range(node_count)]
    return degrees

In [3]:
files =os.listdir('./data')
[f'{i}. {f}' for i,f in enumerate(files)]

['0. gc_1000_1',
 '1. gc_1000_3',
 '2. gc_1000_5',
 '3. gc_1000_7',
 '4. gc_1000_9',
 '5. gc_100_1',
 '6. gc_100_3',
 '7. gc_100_5',
 '8. gc_100_7',
 '9. gc_100_9',
 '10. gc_20_1',
 '11. gc_20_3',
 '12. gc_20_5',
 '13. gc_20_7',
 '14. gc_20_9',
 '15. gc_250_1',
 '16. gc_250_3',
 '17. gc_250_5',
 '18. gc_250_7',
 '19. gc_250_9',
 '20. gc_4_1',
 '21. gc_500_1',
 '22. gc_500_3',
 '23. gc_500_5',
 '24. gc_500_7',
 '25. gc_500_9',
 '26. gc_50_1',
 '27. gc_50_3',
 '28. gc_50_5',
 '29. gc_50_7',
 '30. gc_50_9',
 '31. gc_70_1',
 '32. gc_70_3',
 '33. gc_70_5',
 '34. gc_70_7',
 '35. gc_70_9']

In [57]:
idx = 34
print(files[idx])
with open(f'./data/{files[idx]}','r') as fp:
    input_data = fp.read()
print(input_data[:25])

gc_70_7
70 1678
0 1
0 3
0 4
0 5
0


In [58]:
lines = input_data.split('\n')

first_line = lines[0].split()
n_points = node_count = int(first_line[0])
edge_count = int(first_line[1])

edges = []
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))
    
print(node_count, edge_count)

70 1678


In [59]:
def choose_color(colors):
    color = 0
    while color in colors:
        color +=1
    return color

In [60]:
def greedy_solver(edges, n_points):
    degrees = get_degrees(edges, n_points)
    edge_dict = {i: set() for i in range(n_points)}
    for v1, v2 in edges:
        edge_dict[v1].add(v2)
        edge_dict[v2].add(v1)
    # get indices by degree of the vertex
    idx = np.argsort(degrees)[::-1]
    colors = -np.ones(n_points, dtype=int)
    colors[idx[0]] = 0
    for i, ind in enumerate(idx):
        connected_nodes = edge_dict[ind]
        c = choose_color(colors[list(connected_nodes)])
        colors[ind] = c
    return colors

def semi_greedy_solver(edges,n_points,degrees, edge_dict, order = None):
        
    # get indices by degree of the vertex
    if order is None:
        idx = np.argsort(degrees)[::-1]
    else:
        idx = np.array(order)
    colors = -np.ones(n_points, dtype=int)
    colors[idx[0]] = 0
    for i, ind in enumerate(idx):
        connected_nodes = edge_dict[ind]
        c = choose_color(colors[list(connected_nodes)])
        colors[ind] = c
    return colors

# Random Selection

In [61]:
def swap_elements(array,n=1,random_state = None):
    np.random.seed(random_state)
    x = np.array(array)
    length = len(x)
    for _ in range(n):
        i,j = np.random.choice(range(length),size=2,replace = False)
        x[i], x[j] = x[j], x[i]
    return x

In [24]:
N_ROUNDS = 1000

degrees = get_degrees(edges, n_points)
edge_dict = {i: set() for i in range(n_points)}
for v1, v2 in edges:
    edge_dict[v1].add(v2)
    edge_dict[v2].add(v1)

solution = greedy_solver(edges, n_points)
n_colors = max(solution) + 1
idx = np.argsort(degrees)[::-1]

pbar = trange(N_ROUNDS)
for i in pbar:
    n = np.random.randint(1, 6)
    new_idx = swap_elements(idx.copy(), n)
    res = semi_greedy_solver(
        edges,
        n_points,
        degrees,
        edge_dict,
        order=new_idx,
    )
    res_colors = np.max(res) + 1

    desc = f'#{i+1:03.0f}:  {res_colors}  / {n_colors}\n'

    pbar.set_description(desc, refresh=True)
    if res_colors <= n_colors:
        solution = res
        n_colors = res_colors
        idx = new_idx

# Genetic Algorithm

In [30]:
from collections import namedtuple
from copy import deepcopy

In [63]:
GreedyGraph = namedtuple('GreedyGraph',['colors','order','count'])

In [64]:
degrees = get_degrees(edges, n_points)
edge_dict = {i: set() for i in range(n_points)}
for v1, v2 in edges:
    edge_dict[v1].add(v2)
    edge_dict[v2].add(v1)

solution = greedy_solver(edges, n_points)
n_colors = max(solution) + 1
idx = np.argsort(degrees)[::-1]
print(n_colors)

22


In [65]:
np.random.seed(1)
# Initialize population
N_POP = 100
N_ROUNDS = 200
population = [GreedyGraph(solution,idx,n_colors)]

for _ in range(1,N_POP):
    n = np.random.randint(1, 6)
    new_idx = swap_elements(idx.copy(), n)
    res = semi_greedy_solver(
        edges,
        n_points,
        degrees,
        edge_dict,
        order=new_idx,
    )
    res_colors = np.max(res) + 1
    population.append(GreedyGraph(res,new_idx,res_colors))

In [67]:
for i in tqdm(range(N_ROUNDS)):
    best = np.min([g.count for g in population])
    worst = np.max([g.count for g in population])
    print(f'#{i:03.0f}\tBest: {best}  \tWorst: {worst}  \t {len(population)}')
    if best==worst:
        new_population = [deepcopy(g) for g in population if np.random.rand()>.75]
    else:
        new_population = [deepcopy(g) for g in population if g.count==best]
    while len(new_population)<N_POP:#for _ in range(len(new_popultaion),N_POP):
        selected = population[np.random.randint(N_POP)]
        n = np.random.randint(1,6)
        new_idx = swap_elements(selected.order,n)
        res = semi_greedy_solver(
            edges,
            n_points,
            degrees,
            edge_dict,
            order=new_idx,
        )
        res_colors = np.max(res) + 1
        new_population.append(GreedyGraph(res,new_idx,res_colors))
    population = new_population.copy()
    

#000	Best: 18  	Worst: 21  	 100
#001	Best: 18  	Worst: 19  	 100
#002	Best: 18  	Worst: 21  	 100
#003	Best: 18  	Worst: 20  	 100
#004	Best: 18  	Worst: 20  	 100
#005	Best: 18  	Worst: 20  	 100
#006	Best: 18  	Worst: 19  	 100
#007	Best: 18  	Worst: 18  	 100
#008	Best: 18  	Worst: 23  	 100
#009	Best: 18  	Worst: 24  	 100
#010	Best: 18  	Worst: 24  	 100
#011	Best: 18  	Worst: 24  	 100
#012	Best: 18  	Worst: 24  	 100
#013	Best: 18  	Worst: 23  	 100
#014	Best: 18  	Worst: 23  	 100
#015	Best: 18  	Worst: 25  	 100
#016	Best: 18  	Worst: 23  	 100
#017	Best: 18  	Worst: 23  	 100
#018	Best: 18  	Worst: 24  	 100
#019	Best: 18  	Worst: 24  	 100
#020	Best: 18  	Worst: 23  	 100
#021	Best: 18  	Worst: 24  	 100
#022	Best: 18  	Worst: 23  	 100
#023	Best: 18  	Worst: 23  	 100
#024	Best: 18  	Worst: 24  	 100
#025	Best: 18  	Worst: 23  	 100
#026	Best: 18  	Worst: 23  	 100
#027	Best: 18  	Worst: 23  	 100
#028	Best: 18  	Worst: 24  	 100
#029	Best: 18  	Worst: 23  	 100
#030	Best:

In [70]:
N_POP = 100
N_ROUNDS = 1000

GreedyGraph = namedtuple('GreedyGraph',['colors','order','count'])

def ga_solver(edges, n_points, random_state = None):
    np.random.seed(random_state)
    
   
    # Get degree of each node
    degrees = get_degrees(edges, n_points)
    
    # Create a dictionary of connections for each node
    edge_dict = {i: set() for i in range(n_points)}
    for v1, v2 in edges:
        edge_dict[v1].add(v2)
        edge_dict[v2].add(v1)
    
    # Get initial solution using greedy
    solution = greedy_solver(edges, n_points)
    n_colors = max(solution) + 1
    idx = np.argsort(degrees)[::-1]
    n_greedy = n_colors

    # Genetic Algorithm
    
    # Initialize population
    population = [GreedyGraph(solution,idx,n_colors)]

    for _ in range(1,N_POP):
        n = np.random.randint(1, 6)
        new_idx = swap_elements(idx.copy(), n)
        res = semi_greedy_solver(
            edges,
            n_points,
            degrees,
            edge_dict,
            order=new_idx,
        )
        res_colors = np.max(res) + 1
        population.append(GreedyGraph(res,new_idx,res_colors))
    
    # Evolution
    pbar = trange(N_ROUNDS)
    for i in pbar:
        best = np.min([g.count for g in population])
        worst = np.max([g.count for g in population])
        desc = f'{n_greedy} -> {best}/{worst}'
        pbar.set_description(desc, refresh=True)
        if best==worst:
            new_population = [deepcopy(g) for g in population if np.random.rand()>.75]
        else:
            new_population = [deepcopy(g) for g in population if g.count==best]
        while len(new_population)<N_POP:
            selected = population[np.random.randint(N_POP)]
            n = np.random.randint(1,6)
            new_idx = swap_elements(selected.order,n)
            res = semi_greedy_solver(
                edges,
                n_points,
                degrees,
                edge_dict,
                order=new_idx,
            )
            res_colors = np.max(res) + 1
            new_population.append(GreedyGraph(res,new_idx,res_colors))
        population = new_population.copy()
        
    id_sol = np.argmin([g.count for g in population])
    return population[id_sol].colors

In [71]:
N_ROUNDS=100
ga_solver(edges,n_points,1)

array([ 6,  1,  1, 11,  9, 18,  3,  5,  0,  5, 15, 13,  3, 16,  9,  0,  8,
       10, 11,  2, 12, 13,  9,  7,  4,  9, 16, 11, 12, 17, 15, 14, 13,  5,
       18, 16, 12,  0,  6, 12, 14,  4, 10, 18,  2,  3,  2, 13, 14, 11,  2,
        6,  0,  8, 10,  6, 15,  4,  3, 17,  1,  8, 12,  7,  1,  1, 10, 15,
        7,  4])

# Group rearangement

In [115]:
files =os.listdir('./data')
[f'{i}. {f}' for i,f in enumerate(files)]

['0. gc_1000_1',
 '1. gc_1000_3',
 '2. gc_1000_5',
 '3. gc_1000_7',
 '4. gc_1000_9',
 '5. gc_100_1',
 '6. gc_100_3',
 '7. gc_100_5',
 '8. gc_100_7',
 '9. gc_100_9',
 '10. gc_20_1',
 '11. gc_20_3',
 '12. gc_20_5',
 '13. gc_20_7',
 '14. gc_20_9',
 '15. gc_250_1',
 '16. gc_250_3',
 '17. gc_250_5',
 '18. gc_250_7',
 '19. gc_250_9',
 '20. gc_4_1',
 '21. gc_500_1',
 '22. gc_500_3',
 '23. gc_500_5',
 '24. gc_500_7',
 '25. gc_500_9',
 '26. gc_50_1',
 '27. gc_50_3',
 '28. gc_50_5',
 '29. gc_50_7',
 '30. gc_50_9',
 '31. gc_70_1',
 '32. gc_70_3',
 '33. gc_70_5',
 '34. gc_70_7',
 '35. gc_70_9']

In [164]:
idx = 2
print(files[idx])
with open(f'./data/{files[idx]}','r') as fp:
    input_data = fp.read()
print(input_data[:25])

gc_1000_5
1000 249482
0 1
0 2
0 3
0


In [165]:
lines = input_data.split('\n')

first_line = lines[0].split()
n_points = node_count = int(first_line[0])
edge_count = int(first_line[1])

edges = []
for i in range(1, edge_count + 1):
    line = lines[i]
    parts = line.split()
    edges.append((int(parts[0]), int(parts[1])))
    
print(node_count, edge_count)

1000 249482


In [170]:
def sort_by_degree(all_degrees,idx):
    degrees = np.array([all_degrees[i] for i in idx])
    argsort = np.argsort(degrees)[::-1]
    return np.array(idx)[argsort]

def shuffle_groups(degrees, groups):
    new_groups = [sort_by_degree(degrees, group) for group in groups.copy()]
    np.random.shuffle(new_groups)
    if isinstance(new_groups[0],list):
        new_idx = sum(new_groups,start = [])
    else:
        new_idx = np.concatenate(new_groups)
    return new_idx

def count_colors(colors):
    return len(set(colors))
    

def reordered_greedy(edges, n_points, random_state = None, trials = 2000):
    # Set Seed value
    np.random.seed(random_state)
    
    # Get degree of each node
    degrees = get_degrees(edges, n_points)
    
    # Create a dictionary of connections for each node
    edge_dict = {i: set() for i in range(n_points)}
    for v1, v2 in edges:
        edge_dict[v1].add(v2)
        edge_dict[v2].add(v1)
    
    # Get initial solution using greedy
    solution = greedy_solver(edges, n_points)
    n_colors = count_colors(solution)
    idx = np.argsort(degrees)[::-1]
    n_greedy = n_colors
    
    pbar = trange(trials)
    for i in pbar:
        groups = [np.argwhere(solution==c).flatten() for c in range(n_colors)]
        new_groups = shuffle_groups(degrees,groups)
        solution = semi_greedy_solver(edges,n_points,degrees,edge_dict,new_groups)
        n_colors = count_colors(solution)
        desc = f'#{i+1:04.0f} \t{n_colors}/{n_greedy}'
        pbar.set_description(desc, refresh=True)
        
    return solution
        

In [169]:
reordered_greedy(edges,n_points,1,4000)

KeyboardInterrupt: 

In [130]:
%debug

> <ipython-input-128-802521fa438c>(9)shuffle_groups()
      7     new_groups = [sort_by_degree(degrees, group) for group in groups.copy()]
      8     np.random.shuffle(new_groups)
----> 9     new_idx = sum(new_groups,start = [])
     10     return new_idx
     11 

ipdb> p new_groups
[array([49,  7, 48, 44, 16, 10, 26, 22, 33, 36, 12,  4, 29, 20, 42, 31],
      dtype=int64), array([0], dtype=int64), array([11, 46, 43,  5, 38, 23, 21], dtype=int64), array([47, 19, 40, 32,  2, 25, 18, 45, 39, 24, 13,  6, 34, 28],
      dtype=int64), array([30, 27,  8, 41,  3, 17, 15, 14, 37, 35,  1,  9], dtype=int64)]
ipdb> q


In [135]:
np.concatenate([np.array([1,2]),np.array([3,4,8])])

array([1, 2, 3, 4, 8])